# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manyli

In [2]:
%cd '/content/drive/MyDrive/'

[Errno 2] No such file or directory: '/content/drive/MyDrive/'
/content


In [3]:
!unzip /content/data.zip

unzip:  cannot find or open /content/data.zip, /content/data.zip.zip or /content/data.zip.ZIP.


# *Importing libraries*

In [4]:
import json
import wandb
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")

train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])

In [6]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [7]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [8]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [9]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [10]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
})

In [12]:
import pickle
with open('/content/rh-code-mixed-2.pkl', 'rb') as f:
    related_words = pickle.load(f)

## Adjacent Sentence-Language Approach

In [13]:
def adjacent_preprocess(sentences):
    texts = sentences['clean_text']
    tags = sentences['language_tags']

    processed_texts = []

    for text, tag in zip(texts, tags):
        if isinstance(tag, str):
            tag = tag.split(', ')  # Convert a string of tags into a list of tags

        tag_string = " ".join([f"[{label}]" for label in tag])
        text = " ".join([f"{related_words[word].get('corrected_word',word)}" for word in text.split(" ")])
        adjacent_text = f"{text} {tag_string}"
        processed_texts.append(adjacent_text)

    return {"adjacent_text_tags": processed_texts}

dataset = dataset.map(adjacent_preprocess, batched=True)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [14]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'adjacent_text_tags': 'hindu to kuch he par tum to puri duniya may gandagi machaa rahe ho or bat rahi bhagave kii to tumhe aba ke abe bhagava hi the talwaar ke dar se salwar khol di unhone isamen tumhaari koi galati nahin he [EN] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [EN] [EN] [HI] [HI] [HI] [HI] [HI] [EN] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI]'}

In [15]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'adjacent_text_tags': 'hindu rastra ke rujhaan mia bhai ko cast he chalo hindu is bat pe mast he [EN] [EN] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [EN] [HI] [EN] [HI] [EN] [HI]'}

In [16]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'adjacent_text_tags': 'jai hind , in namoonon ko bat bhi kerani nahin ati he , inako pension chahie [HI] [EN] [OOV] [EN] [HI] [HI] [EN] [HI] [HI] [HI] [HI] [HI] [OOV] [HI] [EN] [HI]'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [17]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))
special_tokens = ["[EN]", "[HI]", "[OOV]"]
tokenizer.add_tokens(special_tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


3

In [18]:
len(tokenizer)

119550

In [19]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['adjacent_text_tags'],max_length=194,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","language_tags","adjacent_text_tags"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [20]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460,  10114,  15694,  10269,  10261,  10248,  13055,
          10147,  10114,  32385,  10116,  23145,  14990,  11387,  13412,  34350,
         109664,  17394,  11859,  11643,  13173,  10345,  11519,  11859,  11924,
            170,  65864,  10612,  10879,  10116,  10114,  13055,  34927,  10112,
          11357,  10113,  11163,  11357,  10112,    170,  65864,  10362,  11520,
          10105,  13675,  69689,  11163,  12923,  10126,  31119,  29445,    179,
          32631,  10120,  10119,  50198,  26531,  10136,  13055,  34927,  21960,
          10116,  11252,  10116,  74288,  10325,  62137,  10115,  10261, 119547,
         119548, 119548, 119548, 119548, 119548, 119548, 119548, 119548, 119548,
         119548, 119548, 119548, 119548, 119547, 119547, 119548, 119548, 119548,
         119548, 119548, 119547, 119548, 119548, 119548, 119548, 119548, 119548,
         119548, 119548, 119548, 119548, 119548, 119548, 119548, 119548, 1

In [21]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', 'to', 'ku', '##ch', 'he', 'par', 'tu', '##m', 'to', 'pur', '##i', 'dun', '##iya', 'may', 'gan', '##dagi', 'mach', '##aa', 'ra', '##he', 'ho', 'or', 'bat', 'ra', '##hi', 'b', '##haga', '##ve', 'ki', '##i', 'to', 'tu', '##mh', '##e', 'ab', '##a', 'ke', 'ab', '##e', 'b', '##haga', '##va', 'hi', 'the', 'tal', '##waar', 'ke', 'dar', 'se', 'sal', '##war', 'k', '##hol', 'di', 'un', '##hone', 'isam', '##en', 'tu', '##mh', '##aar', '##i', 'ko', '##i', 'gala', '##ti', 'nahi', '##n', 'he', '[EN]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[EN]', '[EN]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[EN]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [22]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['adjacent_text_tags']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu to kuch he par tum to puri duniya may gandagi machaa rahe ho or bat rahi bhagave kii to tumhe aba ke abe bhagava hi the talwaar ke dar se salwar khol di unhone isamen tumhaari koi galati nahin he [EN] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [EN] [EN] [HI] [HI] [HI] [HI] [HI] [EN] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI] [HI]
tokenize:              ['hin', '##du', 'to', 'ku', '##ch', 'he', 'par', 'tu', '##m', 'to', 'pur', '##i', 'dun', '##iya', 'may', 'gan', '##dagi', 'mach', '##aa', 'ra', '##he', 'ho', 'or', 'bat', 'ra', '##hi', 'b', '##haga', '##ve', 'ki', '##i', 'to', 'tu', '##mh', '##e', 'ab', '##a', 'ke', 'ab', '##e', 'b', '##haga', '##va', 'hi', 'the', 'tal', '##waar', 'ke', 'dar', 'se', 'sal', '##war', 'k', '##hol', 'di', 'un', '##hone', 'isam', '##en', 'tu', '##mh', '##aar', '##i', 'ko', '##i', 'gala', '##ti', 'nahi', '##n', 'he', '[EN]', '[HI]', '[HI]', '[HI]', '[HI]', '[HI]',

# *Calculating Weights for Labels with Imbalanced Dataset*

In [23]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [24]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [25]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'recall_micro': recall_micro,
        'recall_macro': recall_macro,
        'recall_positive': recall_positive,
        'recall_negative': recall_negative,
        'f1_weighted': f1,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_positive': f1_positive,
        'f1_negative': f1_negative
    }

In [27]:
!export CUDA_LAUNCH_BLOCKING=1

In [28]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

model.resize_token_embeddings(len(tokenizer))

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_atags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)


trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [29]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [30]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_atags/log.jsonl"))

In [31]:
# train the model
wandb.init(project="rh5", name="Hing_mBERT_atag")
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.684850,0.748286,0.693943,0.748286,0.748286,0.543159,0.145749,0.940568,0.697289,0.748286,0.534408,0.218845,0.849971
2,No log,0.541116,0.732615,0.797933,0.732615,0.732615,0.740946,0.757085,0.724806,0.749567,0.732615,0.691177,0.578053,0.804301
3,No log,0.511191,0.726738,0.819988,0.726738,0.726738,0.767392,0.846154,0.688630,0.745910,0.726738,0.696139,0.599713,0.792565
4,No log,0.530331,0.787463,0.804190,0.787463,0.787463,0.745420,0.663968,0.826873,0.793787,0.787463,0.728439,0.601835,0.855043
5,No log,0.653730,0.806072,0.809403,0.806072,0.806072,0.743911,0.623482,0.864341,0.807614,0.806072,0.739895,0.608696,0.871094
6,No log,0.722187,0.803134,0.802864,0.803134,0.803134,0.730947,0.591093,0.870801,0.802998,0.803134,0.731265,0.592292,0.870239
7,No log,0.885688,0.787463,0.790769,0.787463,0.787463,0.717855,0.582996,0.852713,0.789017,0.787463,0.714556,0.570297,0.858816
8,No log,0.895339,0.716944,0.789532,0.716944,0.716944,0.727853,0.748988,0.706718,0.735495,0.716944,0.676245,0.561457,0.791034
9,No log,1.560875,0.787463,0.770118,0.787463,0.787463,0.659966,0.412955,0.906977,0.773822,0.787463,0.675346,0.484561,0.866132


TrainOutput(global_step=432, training_loss=0.3536461724175347, metrics={'train_runtime': 257.9185, 'train_samples_per_second': 118.642, 'train_steps_per_second': 1.861, 'train_loss': 0.3536461724175347, 'epoch': 9.0})

In [32]:
mbert_results = trainer.evaluate()

In [33]:
print(mbert_results)

{'eval_loss': 0.6537297964096069, 'eval_accuracy': 0.8060724779627816, 'eval_precision_weighted': 0.8094028383307845, 'eval_recall_weighted': 0.8060724779627816, 'eval_recall_micro': 0.8060724779627816, 'eval_recall_macro': 0.7439114333239181, 'eval_recall_positive': 0.6234817813765182, 'eval_recall_negative': 0.8643410852713178, 'eval_f1_weighted': 0.8076144844142571, 'eval_f1_micro': 0.8060724779627815, 'eval_f1_macro': 0.7398947010869565, 'eval_f1_positive': 0.6086956521739131, 'eval_f1_negative': 0.87109375, 'eval_runtime': 2.8537, 'eval_samples_per_second': 357.779, 'eval_steps_per_second': 22.427, 'epoch': 9.0}


# **Hing-MBERT Test results**

In [34]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [35]:
test_results_mbert

PredictionOutput(predictions=array([[ 2.1318085 , -2.243535  ],
       [ 0.7162185 , -1.0079604 ],
       [ 2.2558208 , -2.3394248 ],
       ...,
       [-1.1955523 ,  0.7434073 ],
       [ 0.18625426, -0.48317412],
       [ 1.2937502 , -1.4145699 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.6779128909111023, 'test_accuracy': 0.8080313418217434, 'test_precision_weighted': 0.8085640405348716, 'test_recall_weighted': 0.8080313418217434, 'test_recall_micro': 0.8080313418217434, 'test_recall_macro': 0.7396902363242632, 'test_recall_positive': 0.6072874493927125, 'test_recall_negative': 0.872093023255814, 'test_f1_weighted': 0.8082942042177904, 'test_f1_micro': 0.8080313418217434, 'test_f1_macro': 0.7390299628594083, 'test_f1_positive': 0.6048387096774194, 'test_f1_negative': 0.8732212160413971, 'test_runtime': 2.8923, 'test_samples_per_second': 353.004, 'test_steps_per_second': 22.128})

In [36]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 0, 0, ..., 1, 0, 0])

In [37]:
test_df['atags_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [38]:
print(test_results_mbert.metrics)

{'test_loss': 0.6779128909111023, 'test_accuracy': 0.8080313418217434, 'test_precision_weighted': 0.8085640405348716, 'test_recall_weighted': 0.8080313418217434, 'test_recall_micro': 0.8080313418217434, 'test_recall_macro': 0.7396902363242632, 'test_recall_positive': 0.6072874493927125, 'test_recall_negative': 0.872093023255814, 'test_f1_weighted': 0.8082942042177904, 'test_f1_micro': 0.8080313418217434, 'test_f1_macro': 0.7390299628594083, 'test_f1_positive': 0.6048387096774194, 'test_f1_negative': 0.8732212160413971, 'test_runtime': 2.8923, 'test_samples_per_second': 353.004, 'test_steps_per_second': 22.128}


# **Hing BERT Training**

In [39]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [40]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
bert_tokenizer.add_tokens(special_tokens)
bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['adjacent_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","language_tags","adjacent_text_tags"])
bert_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [41]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_atags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_atags/log.jsonl"))

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [42]:
# train the model
wandb.init(project="rh5", name="Hing_BERT_atag")
bert_trainer.train()

eval/accuracy,▃▂▂▇██▇▁▇█
eval/f1_macro,▁▆▇███▇▆▆█
eval/f1_micro,▃▂▂▇██▇▁▇█
eval/f1_negative,▆▂▁▇██▇▁██
eval/f1_positive,▁▇████▇▇▆█
eval/f1_weighted,▁▄▄▇██▇▃▆█
eval/loss,▂▁▁▁▂▂▃▄█▂
eval/precision_weighted,▁▇█▇▇▇▆▆▅▇
eval/recall_macro,▁▇█▇▇▇▆▇▅▇
eval/recall_micro,▃▂▂▇██▇▁▇█
eval/recall_negative,█▂▁▅▆▆▆▂▇▆


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.684392,0.755142,0.713375,0.755142,0.755142,0.568355,0.206478,0.930233,0.716040,0.755142,0.570922,0.289773,0.852071
2,No log,0.549202,0.736533,0.789690,0.736533,0.736533,0.729747,0.716599,0.742894,0.751834,0.736533,0.689324,0.568218,0.810430
3,No log,0.517893,0.711068,0.800440,0.711068,0.711068,0.740517,0.797571,0.683463,0.731133,0.711068,0.676905,0.571843,0.781966
4,No log,0.535045,0.788443,0.807691,0.788443,0.788443,0.751580,0.680162,0.822997,0.795440,0.788443,0.731865,0.608696,0.855034
5,No log,0.692039,0.796278,0.791987,0.796278,0.796278,0.711264,0.546559,0.875969,0.793911,0.796278,0.715931,0.564854,0.867008
6,No log,1.265751,0.803134,0.788992,0.803134,0.803134,0.637223,0.315789,0.958656,0.773363,0.803134,0.658843,0.436975,0.880712
7,No log,0.955320,0.805093,0.811714,0.805093,0.805093,0.750157,0.643725,0.856589,0.807959,0.805093,0.742298,0.615087,0.869508
8,No log,1.089008,0.745348,0.787202,0.745348,0.745348,0.725913,0.688259,0.763566,0.758482,0.745348,0.693181,0.566667,0.819695
9,No log,1.818731,0.809011,0.797595,0.809011,0.809011,0.646612,0.331984,0.961240,0.780761,0.809011,0.670480,0.456825,0.884135
10,No log,0.970579,0.770813,0.806475,0.770813,0.770813,0.753735,0.720648,0.786822,0.781882,0.770813,0.721116,0.603390,0.838843


TrainOutput(global_step=480, training_loss=0.3178755760192871, metrics={'train_runtime': 262.3092, 'train_samples_per_second': 116.656, 'train_steps_per_second': 1.83, 'train_loss': 0.3178755760192871, 'epoch': 10.0})

In [43]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.9553201198577881, 'eval_accuracy': 0.8050930460333007, 'eval_precision_weighted': 0.8117144782760722, 'eval_recall_weighted': 0.8050930460333007, 'eval_recall_micro': 0.8050930460333007, 'eval_recall_macro': 0.7501569218215485, 'eval_recall_positive': 0.6437246963562753, 'eval_recall_negative': 0.8565891472868217, 'eval_f1_weighted': 0.8079587103772548, 'eval_f1_micro': 0.8050930460333007, 'eval_f1_macro': 0.7422976186701336, 'eval_f1_positive': 0.6150870406189556, 'eval_f1_negative': 0.8695081967213114, 'eval_runtime': 2.8236, 'eval_samples_per_second': 361.596, 'eval_steps_per_second': 22.666, 'epoch': 10.0}


# **Test results for Hing-BERT**

In [44]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['atags_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 1.0570652484893799, 'test_accuracy': 0.7639569049951028, 'test_precision_weighted': 0.7682689151986986, 'test_recall_weighted': 0.7639569049951028, 'test_recall_micro': 0.7639569049951028, 'test_recall_macro': 0.6871894255615185, 'test_recall_positive': 0.5384615384615384, 'test_recall_negative': 0.8359173126614987, 'test_f1_weighted': 0.7659835674996134, 'test_f1_micro': 0.7639569049951028, 'test_f1_macro': 0.683825787509075, 'test_f1_positive': 0.52465483234714, 'test_f1_negative': 0.8429967426710098, 'test_runtime': 2.8805, 'test_samples_per_second': 354.456, 'test_steps_per_second': 22.219}


In [45]:
test_df.to_csv('final_test_5k_itags.csv',index=False, encoding='utf-8')

# **Muril cased training**

In [46]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [47]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

## Adding special tokens
special_tokens = ["[EN]", "[HI]", "[OOV]"]
muril_tokenizer.add_tokens(special_tokens)
print(len(muril_tokenizer))
muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['adjacent_text_tags'],padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)

muril_dataset = muril_dataset.remove_columns(["clean_text","language_tags","adjacent_text_tags"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

muril_model.resize_token_embeddings(len(muril_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_atags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    optim = 'adafactor',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to = "wandb"
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_atags/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

197285
197288


Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [48]:
wandb.init(project="rh5", name="muril_atagv1")
muril_trainer.train()

eval/accuracy,▄▃▁▇▇██▃█▅█
eval/f1_macro,▁▆▅█▇▅█▆▅▇█
eval/f1_micro,▄▃▁▇▇██▃█▅█
eval/f1_negative,▆▃▁▆▇█▇▄█▅▇
eval/f1_positive,▁▇▇█▇▄█▇▅██
eval/f1_weighted,▁▄▂▇▇▅█▄▆▆█
eval/loss,▂▁▁▁▂▅▃▄█▃▃
eval/precision_weighted,▁▆▇█▇▆█▆▇██
eval/recall_macro,▁▇██▆▄█▇▄██
eval/recall_micro,▄▃▁▇▇██▃█▅█
eval/recall_negative,▇▂▁▅▆█▅▃█▄▅


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.693070,0.757101,0.574506,0.757101,0.757101,0.499354,0.000000,0.998708,0.653284,0.757101,0.430881,0.000000,0.861761
2,No log,0.692965,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
3,No log,0.693037,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
4,No log,0.693056,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
5,No log,0.693034,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
6,No log,0.692966,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

TrainOutput(global_step=288, training_loss=0.6931383874681261, metrics={'train_runtime': 403.5443, 'train_samples_per_second': 151.656, 'train_steps_per_second': 2.379, 'train_loss': 0.6931383874681261, 'epoch': 6.0})

In [49]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.6929646730422974, 'eval_accuracy': 0.7580803134182175, 'eval_precision_weighted': 0.5746857615922628, 'eval_recall_weighted': 0.7580803134182175, 'eval_recall_micro': 0.7580803134182175, 'eval_recall_macro': 0.5, 'eval_recall_positive': 0.0, 'eval_recall_negative': 1.0, 'eval_f1_weighted': 0.6537650836609474, 'eval_f1_micro': 0.7580803134182175, 'eval_f1_macro': 0.43119777158774375, 'eval_f1_positive': 0.0, 'eval_f1_negative': 0.8623955431754875, 'eval_runtime': 6.7783, 'eval_samples_per_second': 150.627, 'eval_steps_per_second': 9.442, 'epoch': 6.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['atags_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.6928778886795044, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.5746857615922628, 'test_recall_weighted': 0.7580803134182175, 'test_recall_micro': 0.7580803134182175, 'test_recall_macro': 0.5, 'test_recall_positive': 0.0, 'test_recall_negative': 1.0, 'test_f1_weighted': 0.6537650836609474, 'test_f1_micro': 0.7580803134182175, 'test_f1_macro': 0.43119777158774375, 'test_f1_positive': 0.0, 'test_f1_negative': 0.8623955431754875, 'test_runtime': 6.7494, 'test_samples_per_second': 151.272, 'test_steps_per_second': 9.482}


## XLMR training

In [51]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [52]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
xlmr_tokenizer.add_tokens(special_tokens)

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['adjacent_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","language_tags","adjacent_text_tags"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

xlmr_model.resize_token_embeddings(len(xlmr_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_atags",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    optim = 'adafactor',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_atags/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [53]:
import torch
torch.cuda.empty_cache()

In [54]:
wandb.init(project="rh5", name="xlmr_atag3")
xlmr_trainer.train()

eval/accuracy,▁██████
eval/f1_macro,▁██████
eval/f1_micro,▁██████
eval/f1_negative,▁██████
eval/f1_positive,▁▁▁▁▁▁▁
eval/f1_weighted,▁██████
eval/loss,█▁▆▇▆▁▁
eval/precision_weighted,▁██████
eval/recall_macro,▁██████
eval/recall_micro,▁██████
eval/recall_negative,▁██████


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.681728,0.751224,0.703939,0.751224,0.751224,0.556123,0.178138,0.934109,0.707063,0.751224,0.553949,0.257310,0.850588
2,No log,0.664371,0.687561,0.712746,0.687561,0.687561,0.614749,0.473684,0.755814,0.698038,0.687561,0.604454,0.423146,0.785762
3,0.695700,0.658328,0.747307,0.739796,0.747307,0.747307,0.640372,0.433198,0.847545,0.743188,0.747307,0.644529,0.453390,0.835669
4,0.695700,0.674115,0.678746,0.769252,0.678746,0.678746,0.695768,0.728745,0.662791,0.701025,0.678746,0.640505,0.523256,0.757755
5,0.695700,0.596568,0.666014,0.776057,0.666014,0.666014,0.701153,0.769231,0.633075,0.689894,0.666014,0.634454,0.527046,0.741862
6,0.639300,0.562239,0.700294,0.788539,0.700294,0.700294,0.723763,0.769231,0.678295,0.721017,0.700294,0.664136,0.553936,0.774336
7,0.639300,0.584200,0.782566,0.790132,0.782566,0.782566,0.720138,0.599190,0.841085,0.785891,0.782566,0.712880,0.571429,0.854331
8,0.561700,0.598342,0.760039,0.791748,0.760039,0.760039,0.731468,0.676113,0.786822,0.770682,0.760039,0.704696,0.576857,0.832536
9,0.561700,0.596623,0.742409,0.788568,0.742409,0.742409,0.728109,0.700405,0.755814,0.756394,0.742409,0.692307,0.568144,0.816469
10,0.561700,0.822190,0.780607,0.770050,0.780607,0.780607,0.674740,0.469636,0.879845,0.774094,0.780607,0.683768,0.508772,0.858764


TrainOutput(global_step=2304, training_loss=0.5697843399312761, metrics={'train_runtime': 507.4061, 'train_samples_per_second': 120.613, 'train_steps_per_second': 7.568, 'train_loss': 0.5697843399312761, 'epoch': 12.0})

In [55]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.584200382232666, 'eval_accuracy': 0.7825661116552399, 'eval_precision_weighted': 0.79013221246806, 'eval_recall_weighted': 0.7825661116552399, 'eval_recall_micro': 0.7825661116552399, 'eval_recall_macro': 0.7201377773593196, 'eval_recall_positive': 0.5991902834008097, 'eval_recall_negative': 0.8410852713178295, 'eval_f1_weighted': 0.7858911122887309, 'eval_f1_micro': 0.7825661116552399, 'eval_f1_macro': 0.7128796400449944, 'eval_f1_positive': 0.5714285714285714, 'eval_f1_negative': 0.8543307086614174, 'eval_runtime': 2.9884, 'eval_samples_per_second': 341.657, 'eval_steps_per_second': 21.416, 'epoch': 12.0}


In [56]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['atags_predicted_labels_5k_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 0.6301141977310181, 'test_accuracy': 0.7473065621939275, 'test_precision_weighted': 0.7610714843361864, 'test_recall_weighted': 0.7473065621939275, 'test_recall_micro': 0.7473065621939275, 'test_recall_macro': 0.6817207000805532, 'test_recall_positive': 0.5546558704453441, 'test_recall_negative': 0.8087855297157622, 'test_f1_weighted': 0.7531517415263385, 'test_f1_micro': 0.7473065621939277, 'test_f1_macro': 0.6720883334163222, 'test_f1_positive': 0.5150375939849624, 'test_f1_negative': 0.8291390728476821, 'test_runtime': 3.0722, 'test_samples_per_second': 332.337, 'test_steps_per_second': 20.832}


In [57]:
test_df.to_csv('final_atags.csv',index=False, encoding='utf-8')